## 預測價格及區間

In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# --------------------------
# 1. 讀取資料
# --------------------------
df = pd.read_csv('/Users/yuchingchen/Documents/專題/merge_and_cleaned/final_data/long_flight.csv')

# 預先分經濟艙、商務艙
eco_raw = df[df['艙等（主航段）'] == '經濟艙'].copy()
biz_raw = df[df['艙等（主航段）'] == '商務艙'].copy()

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols = [
    '出發時段', '出發機場代號', '抵達時段', '航空聯盟組合', '航空公司組合',
    '航空聯盟', '停靠站數量', '機型分類', '假期', '飛行時間兩段分類', '是否為平日'
]
num_cols = ['停留時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
group_col = '抵達機場代號'
log_col = '平均價格_log'
true_col = '平均價格'

# --------------------------
# 3. 編碼與標準化
# --------------------------
def encode_and_scale(df_raw):
    cols = cat_cols + num_cols + [log_col, true_col]
    df_sub = df_raw[cols]
    df_enc = pd.get_dummies(df_sub, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# 找共同特徵
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for col in [log_col, true_col]:
    if col in common_feats:
        common_feats.remove(col)

# --------------------------
# 4. 拆分訓練集 & 訓練模型
# --------------------------
def split_and_train(df_enc, seed=123):
    X = df_enc[common_feats]
    y = df_enc[log_col]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed
    )
    model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"🔹測試集 MSE = {mean_squared_error(y_test, y_pred):.4f}, R2 = {r2_score(y_test, y_pred):.4f}")
    return model, X_train, y_train, X_test

eco_model, eco_X_train, eco_y_train, eco_X_test = split_and_train(eco_enc)
biz_model, biz_X_train, biz_y_train, biz_X_test = split_and_train(biz_enc)

# --------------------------
# 5. 測試集資料：擷取組合
# --------------------------
def get_test_combinations(df_raw, code):
    cols = cat_cols + num_cols + [true_col]
    return df_raw[df_raw[group_col] == code][cols].drop_duplicates().reset_index(drop=True)

# --------------------------
# 6. 平行 Bootstrap 單次訓練
# --------------------------
def train_and_predict_bootstrap(i, X_train, y_train, X_test):
    boot_idx = np.random.choice(len(X_train), len(X_train), replace=True)
    X_boot = X_train.iloc[boot_idx]
    y_boot = y_train.iloc[boot_idx]
    model = XGBRegressor(n_estimators=100, random_state=123+i, n_jobs=-1)
    model.fit(X_boot, y_boot)
    preds = model.predict(X_test)
    return preds

# --------------------------
# 7. 預測價格 + 預測區間並存在一起
# --------------------------
def predict_and_save_testset_with_interval(raw_df, model, X_train, y_train, X_test, scaler, prefix, n_bootstrap=100):
    out_dir = 'predict_data/long'
    os.makedirs(out_dir, exist_ok=True)

    test_index = X_test.index
    test_df = raw_df.loc[test_index]
    airports = sorted(test_df[group_col].unique())

    # 計算 sigma²（測試集誤差變異數）
    log_preds_test = model.predict(X_test)
    sigma2 = np.var(log_preds_test - np.log1p(test_df[true_col]))

    for code in airports:
        combos = get_test_combinations(test_df, code)
        if combos.empty:
            continue

        # One-hot encode & scale
        sub = combos[cat_cols + num_cols]
        enc = pd.get_dummies(sub, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        enc[num_cols] = scaler.transform(combos[num_cols])

        # 預測平均價格
        log_preds = model.predict(enc)
        combos['預測_平均價格'] = np.expm1(log_preds + 0.5 * sigma2)

        # Bootstrap 預測區間
        results = Parallel(n_jobs=-1)(
            delayed(train_and_predict_bootstrap)(i, X_train, y_train, enc)
            for i in range(n_bootstrap)
        )
        y_preds_bootstrap = np.array(results)

        preds_median = np.expm1(np.median(y_preds_bootstrap, axis=0))
        preds_lower = np.expm1(np.percentile(y_preds_bootstrap, 2.5, axis=0))
        preds_upper = np.expm1(np.percentile(y_preds_bootstrap, 97.5, axis=0))

        # 加上預測區間
        combos['預測中位數'] = preds_median
        combos['預測區間下界'] = preds_lower
        combos['預測區間上界'] = preds_upper
        combos[group_col] = code

        combos = combos.drop_duplicates(subset=cat_cols + num_cols, keep='last')

        # 儲存
        fn = os.path.join(out_dir, f"{prefix}_{code}.csv")
        combos.to_csv(fn, index=False)
        print(f"✅ 預測結果已存：{fn}（共 {len(combos)} 筆）")

# --------------------------
# 8. 執行
# --------------------------
predict_and_save_testset_with_interval(eco_raw, eco_model, eco_X_train, eco_y_train, eco_X_test, eco_scaler, prefix='eco', n_bootstrap=100)
predict_and_save_testset_with_interval(biz_raw, biz_model, biz_X_train, biz_y_train, biz_X_test, biz_scaler, prefix='biz', n_bootstrap=100)

🔹測試集 MSE = 0.0319, R2 = 0.8960
🔹測試集 MSE = 0.0165, R2 = 0.8761
✅ 預測結果已存：predict_data/long/eco_CDG.csv（共 1576 筆）
✅ 預測結果已存：predict_data/long/eco_FRA.csv（共 2120 筆）
✅ 預測結果已存：predict_data/long/eco_JFK.csv（共 1221 筆）
✅ 預測結果已存：predict_data/long/eco_LAX.csv（共 2293 筆）
✅ 預測結果已存：predict_data/long/eco_LHR.csv（共 2644 筆）
✅ 預測結果已存：predict_data/long/eco_SYD.csv（共 1986 筆）
✅ 預測結果已存：predict_data/long/eco_ZRH.csv（共 1367 筆）
✅ 預測結果已存：predict_data/long/biz_CDG.csv（共 1271 筆）
✅ 預測結果已存：predict_data/long/biz_FRA.csv（共 1675 筆）
✅ 預測結果已存：predict_data/long/biz_JFK.csv（共 1235 筆）
✅ 預測結果已存：predict_data/long/biz_LAX.csv（共 2240 筆）
✅ 預測結果已存：predict_data/long/biz_LHR.csv（共 2003 筆）
✅ 預測結果已存：predict_data/long/biz_SYD.csv（共 1603 筆）
✅ 預測結果已存：predict_data/long/biz_ZRH.csv（共 1096 筆）


In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# --------------------------
# 1. 讀取資料
# --------------------------
df = pd.read_csv('/Users/yuchingchen/Documents/專題/merge_and_cleaned/final_data/long_flight.csv')

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols  = [
    '出發時段','出發機場代號','抵達時段',
    '航空聯盟組合','航空公司組合','航空聯盟',
    '停靠站數量','機型分類','假期',
    '飛行時間兩段分類','是否為平日'
]
group_col = '抵達機場代號'
num_cols  = [
    '停留時間_分鐘','實際飛行時間_分鐘',
    '經濟指標','機場指標','competing_flights'
]
log_col   = '平均價格_log'
true_col  = '平均價格'
seed      = 123

# --------------------------
# 3. 分艙等並標記真實價格
# --------------------------
eco_raw = df[df['艙等（主航段）']=='經濟艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()
biz_raw = df[df['艙等（主航段）']=='商務艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()

# --------------------------
# 4. 編碼與標準化
# --------------------------
def encode_and_scale(df_raw):
    df_enc = pd.get_dummies(df_raw, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# --------------------------
# 5. 取共同特徵欄位
# --------------------------
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for col in [log_col, true_col, group_col]:
    if col in common_feats:
        common_feats.remove(col)

# --------------------------
# 6. 最佳參數（長程平均價格_log，全用 XGBoost）
# --------------------------
best_params_avg = {
    '經濟艙': {
        'XGBoost': {
            'n_estimators': 196,
            'learning_rate': 0.1369319380373383,
            'max_depth': 9,
            'subsample': 0.831877718809044,
            'colsample_bytree': 0.8877875879142252
        }
    },
    '商務艙': {
        'XGBoost': {
            'n_estimators': 196,
            'learning_rate': 0.1369319380373383,
            'max_depth': 9,
            'subsample': 0.831877718809044,
            'colsample_bytree': 0.8877875879142252
        }
    }
}

# --------------------------
# 7. 訓練模型 & 拆分測試集（全用 XGBoost）
# --------------------------
def split_and_train(df_enc, cabin_type):
    X = df_enc[common_feats]
    y = df_enc[log_col]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed
    )
    params = best_params_avg[cabin_type]['XGBoost']
    model = XGBRegressor(
        tree_method='hist',
        verbosity=0,
        random_state=seed,
        n_jobs=-1,
        **params
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"🔹[{cabin_type}] 測試集 MSE = {mean_squared_error(y_test, y_pred):.4f}, "
          f"R2 = {r2_score(y_test, y_pred):.4f}")
    return model, X_train, y_train, X_test

eco_model, eco_X_train, eco_y_train, eco_X_test = split_and_train(eco_enc, '經濟艙')
biz_model, biz_X_train, biz_y_train, biz_X_test = split_and_train(biz_enc, '商務艙')

# --------------------------
# 8. Bootstrap 單次訓練（全用 XGBoost）
# --------------------------
def train_and_predict_bootstrap(i, X_train, y_train, X_test, cabin_type):
    idx = np.random.choice(len(X_train), len(X_train), replace=True)
    Xb, yb = X_train.iloc[idx], y_train.iloc[idx]
    params = best_params_avg[cabin_type]['XGBoost']
    model = XGBRegressor(
        tree_method='hist',
        verbosity=0,
        random_state=seed + i,
        n_jobs=-1,
        **params
    )
    model.fit(Xb, yb)
    return model.predict(X_test)

# --------------------------
# 9. 預測 & 儲存帶區間結果
# --------------------------
def predict_and_save_with_interval(
    raw_df, model, X_train, y_train, X_test, scaler, cabin_type, prefix, n_boot=100
):
    out_dir = f'predict_data/long'
    os.makedirs(out_dir, exist_ok=True)
    idx_test = X_test.index
    test_df  = raw_df.loc[idx_test]
    sigma2   = np.var(model.predict(X_test) - np.log1p(test_df[true_col]))

    for code in sorted(test_df[group_col].unique()):
        combo = test_df[test_df[group_col]==code][cat_cols+num_cols+[true_col]] \
                .drop_duplicates().reset_index(drop=True)
        if combo.empty: continue

        enc = pd.get_dummies(combo, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        enc[num_cols] = scaler.transform(combo[num_cols])

        lp = model.predict(enc)
        combo['預測_平均價格'] = np.expm1(lp + 0.5*sigma2)

        boots = Parallel(n_jobs=-1)(
            delayed(train_and_predict_bootstrap)(
                i, X_train, y_train, enc, cabin_type
            ) for i in range(n_boot)
        )
        arr = np.array(boots)

        combo['預測中位數']   = np.expm1(np.median(arr, axis=0))
        combo['預測區間下界'] = np.expm1(np.percentile(arr,2.5,axis=0))
        combo['預測區間上界'] = np.expm1(np.percentile(arr,97.5,axis=0))
        combo[group_col] = code

        fn = f'{out_dir}/{prefix}_{code}.csv'
        combo.to_csv(fn, index=False)
        print(f"✅[{cabin_type}] 已存：{fn} ({len(combo)} 筆)")

predict_and_save_with_interval(
    eco_raw, eco_model, eco_X_train, eco_y_train, eco_X_test,
    eco_scaler, '經濟艙', 'eco', n_boot=100
)
predict_and_save_with_interval(
    biz_raw, biz_model, biz_X_train, biz_y_train, biz_X_test,
    biz_scaler, '商務艙', 'biz', n_boot=100
)

🔹[經濟艙] 測試集 MSE = 0.0267, R2 = 0.9129
🔹[商務艙] 測試集 MSE = 0.0143, R2 = 0.8928
✅[經濟艙] 已存：predict_data/long/eco_CDG.csv (1903 筆)
✅[經濟艙] 已存：predict_data/long/eco_FRA.csv (2347 筆)
✅[經濟艙] 已存：predict_data/long/eco_JFK.csv (1361 筆)
✅[經濟艙] 已存：predict_data/long/eco_LAX.csv (2603 筆)
✅[經濟艙] 已存：predict_data/long/eco_LHR.csv (2949 筆)
✅[經濟艙] 已存：predict_data/long/eco_SYD.csv (2269 筆)
✅[經濟艙] 已存：predict_data/long/eco_ZRH.csv (1623 筆)
✅[商務艙] 已存：predict_data/long/biz_CDG.csv (1499 筆)
✅[商務艙] 已存：predict_data/long/biz_FRA.csv (1841 筆)
✅[商務艙] 已存：predict_data/long/biz_JFK.csv (1392 筆)
✅[商務艙] 已存：predict_data/long/biz_LAX.csv (2561 筆)
✅[商務艙] 已存：predict_data/long/biz_LHR.csv (2260 筆)
✅[商務艙] 已存：predict_data/long/biz_SYD.csv (1826 筆)
✅[商務艙] 已存：predict_data/long/biz_ZRH.csv (1303 筆)
